# Create interface with minimal energy among surfaces with different Miller indices

This notebook takes two materials, cycles through Miller indices and constructs a coherent interface using ZSL algorithm, optimizes it with a relaxer and calculator, and then calculates the delta energy for the interface. The delta energy is then plotted as a function of the Miller index.

## 1. Setup
Set the maximum Miller indices for the substrate, and the settings for the interface, relaxation, and ZSL algorithm.

In [ ]:
MAX_MILLER = {
    "H": 1,
    "K": 1,
    "L": 1,
}

SETTINGS = {
    "SUBSTRATE_PARAMETERS": {
        "MATERIAL_INDEX": 0,  # the index of the material in the materials_in list
        "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane
        "THICKNESS": 3,  # in layers
    },
    "LAYER_PARAMETERS": {
        "MATERIAL_INDEX": 1,  # the index of the material in the materials_in list
        "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
        "THICKNESS": 1,  # in layers
    },
    "INTERFACE_PARAMETERS": {
        "DISTANCE_Z": 3.0,  # in Angstroms
    },
    "ZSL_PARAMETERS": {
        "MAX_AREA": 81,  # The area to consider in Angstrom^2
        "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
        "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
        "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
        "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
    },
    "RELAXATION_PARAMETERS": {
        "RELAXER": "BFGS",  # The relaxation algorithm to use
        "CALCULATOR": "EMT",  # The calculator to use
        "FMAX": 0.5,  # The maximum force allowed on each atom
    }
}


## 2. Definitions
Functions definitions to create the interfaces, sort them, relax them, and calculate the effective delta energy.

In [ ]:
from src.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from src.utils import to_pymatgen
from src.utils import ase_to_poscar, pymatgen_to_ase
from ase.optimize import BFGS
from ase.calculators.emt import EMT

OPTIMIZER_MAP = [{"BFGS": BFGS}]
CALCULATOR_MAP = [{"EMT": EMT()}]


def create_interfaces(materials, settings):
    """Create interfaces for the given materials and settings."""
    print("Creating interfaces...")
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]],
        film_structure=materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Find terminations
    cib._find_terminations()
    terminations = cib.terminations

    # Create interfaces for each termination
    interfaces = {}
    for termination in terminations:
        interfaces[termination] = []
        for interface in cib.get_interfaces(
                termination,
                gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
                film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
                substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
                in_layers=True,
        ):
            # Wrap atoms to unit cell
            interface["interface"].make_supercell((1, 1, 1), to_unit_cell=True)
            interfaces[termination].append(interface)
    return interfaces, terminations


# Sort interfaces by the specified strain mode and number of sites
def sort_interfaces(interfaces, terminations, strain_mode="mean_abs_strain"):
    sorted_interfaces = {}
    for termination in terminations:
        sorted_interfaces[termination] = sorted(
            interfaces[termination], key=lambda x: (x[strain_mode], x["interface"].num_sites)
        )
    return sorted_interfaces


def relax_structure(atoms, optimizer, calculator, fmax=0.05, verbose=False):
    # Set up the interface for relaxation
    ase_structure = atoms.copy()
    ase_structure.set_calculator(calculator)
    dyn = optimizer(ase_structure)
    dyn.run(fmax=fmax)

    # Extract results
    ase_original_structure = atoms
    ase_original_structure.set_calculator(calculator)
    ase_final_structure = ase_structure
    relaxed_energy = ase_structure.get_total_energy()

    # Print out the final relaxed structure and energy
    if verbose:
        print('Original structure:\n', ase_to_poscar(ase_original_structure))
        print('\nRelaxed structure:\n', ase_to_poscar(ase_final_structure))
        print(f"The final energy is {float(relaxed_energy):.3f} eV.")

    return ase_final_structure, relaxed_energy


def filter_atoms_by_tag(atoms, material_index):
    """Filter atoms by their tag, corresponding to the material index."""
    return atoms[atoms.get_tags() == material_index]


def get_effective_delta_energy(interface, substrate, layer, calculator):
    """Get the effective energy of the atoms."""
    substrate.set_calculator(calculator)
    layer.set_calculator(calculator)
    interface.set_calculator(calculator)

    relaxed_energy = interface.get_total_energy()
    effective_substrate_energy = substrate.get_total_energy() / substrate.get_global_number_of_atoms()
    effective_layer_energy = layer.get_total_energy() / layer.get_global_number_of_atoms()
    effective_relaxed_energy = relaxed_energy / interface.get_global_number_of_atoms()
    print(f"Bulk substrate energy per atom: {effective_substrate_energy:.3f} eV")
    print(f"Free layer energy per atom: {effective_layer_energy:.3f} eV")
    print(f"Interface energy per atom: {effective_relaxed_energy:.3f} eV")

    # Calculate the effective interface energy

    # number of atoms composing the interface: total, substrate's, layer's
    number_of_interface_atoms = interface.get_global_number_of_atoms()
    number_of_substrate_atoms = filter_atoms_by_tag(interface, SETTINGS["SUBSTRATE_PARAMETERS"][
        "MATERIAL_INDEX"]).get_global_number_of_atoms()
    number_of_layer_atoms = filter_atoms_by_tag(interface, SETTINGS["SUBSTRATE_PARAMETERS"][
        "MATERIAL_INDEX"]).get_global_number_of_atoms()
    area = interface.get_volume() / interface.cell[2, 2]

    # The formula is given by: (E_interface - E_substrate_bulk * N_substrate - E_layer_free * N_layer) / (2 * A * N_interface), where A is the area of the interface, and N_interface = N_substrate + N_layer
    # Derived from: https://www.sciencedirect.com/science/article/pii/S2589152922000485
    effective_interface_delta_energy = (
                                               relaxed_energy * number_of_interface_atoms - effective_substrate_energy * number_of_substrate_atoms - effective_layer_energy * number_of_layer_atoms) / (
                                               2 * area * number_of_interface_atoms)

    print(
        f"Effective interface delta energy: {effective_interface_delta_energy:.3f} eV/Å^2 ({effective_interface_delta_energy / 0.16:.3f} J/m^2)")
    return effective_interface_delta_energy


def create_relaxed_interface_with_min_strain(materials, settings, optimizer, calculator):
    """Creates an interface with minimal strain using ZSL and relaxes it using specified optimizer and calculator."""
    interfaces, terminations = create_interfaces(
        materials=materials,
        settings=settings
    )
    sorted_interfaces = sort_interfaces(interfaces, terminations)
    # TODO: try for every termination
    termination_index = 0
    interface_index = 0
    termination = terminations[termination_index]
    interface = sorted_interfaces[termination][interface_index]["interface"]

    ase_interface = pymatgen_to_ase(interface)
    ase_final_interface, relaxed_energy = relax_structure(
        atoms=ase_interface,
        optimizer=optimizer,
        calculator=calculator,
        fmax=settings["RELAXATION_PARAMETERS"]["FMAX"],
        verbose=True  # Assuming you want to print details
    )

    # Corrected to pass ASE Atoms objects after converting from Pymatgen structures
    ase_substrate = pymatgen_to_ase(materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]])
    ase_layer = pymatgen_to_ase(materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]])

    effective_delta_energy = get_effective_delta_energy(
        interface=ase_final_interface,
        substrate=ase_substrate,
        layer=ase_layer,
        calculator=calculator
    )

    return ase_final_interface, relaxed_energy, effective_delta_energy

## 3. Import materials
From the materials_list folder import the materials in ESSE format.

In [ ]:
import os
import json

materials_in = []
input_folder = "materials_list"
for file in os.listdir(input_folder):
    with open(f"{input_folder}/{file}", "r") as f:
        data = f.read()
        materials_in.append(json.loads(data))
        
if "materials_in" in globals():
    pymatgen_materials = [to_pymatgen(item) for item in materials_in]
for material in pymatgen_materials:
    print(material, "\n")


## 4. Create coherent interfaces with minimal energy among surfaces with different Miller
Cycle through HKL indices to create all matching interfaces with minimal strain

In [ ]:
optimizer = OPTIMIZER_MAP[SETTINGS["RELAXATION_PARAMETERS"]["RELAXER"]]
calculator = CALCULATOR_MAP[SETTINGS["RELAXATION_PARAMETERS"]["CALCULATOR"]]

delta_energies = []
# Cycle through HKL indices for substrate and (0,0,1) for layer
for h in range(0, MAX_MILLER["H"] + 1):
    for k in range(0, MAX_MILLER["K"] + 1):
        for l in range(0, MAX_MILLER["L"] + 1):
            if h == 0 and k == 0 and l == 0:
                continue
            SETTINGS["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"] = (h, k, l)

            print(
                f"Creating interface for substrate ({h},{k},{l}) and layer {SETTINGS['LAYER_PARAMETERS']['MILLER_INDICES']}")
            interface, relaxed_energy, effective_delta_energy = create_relaxed_interface_with_min_strain(
                materials=pymatgen_materials,
                settings=SETTINGS,
                optimizer=optimizer,
                calculator=calculator
            )
            print(f"Delta energy: {effective_delta_energy:.3f} eV/Å^2 ({effective_delta_energy / 0.16:.3f} J/m^2)")
            delta_energies.append([(h, k, l), effective_delta_energy])
            

## 5. Plot the results

In [ ]:
print(delta_energies)
import matplotlib.pyplot as plt

x = [f"{h}{k}{l}" for h, k, l in [item[0] for item in delta_energies]]
y = [item[1] for item in delta_energies]
plt.bar(x, y)
plt.xlabel("Miller indices")
plt.ylabel("Delta energy (eV/Å^2)")
plt.title("Delta energy as a function of Miller indices")
plt.show()